## Trying conditional GANs in Faces image



In [1]:
#Import the libraries we will need.
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.contrib.slim as slim
import os
import scipy.misc
import scipy

To preprocess the face image

https://github.com/leblancfg/autocrop/blob/master/autocrop/autocrop.py

http://www.paulvangent.com/2016/04/01/emotion-recognition-with-python-opencv-and-a-face-dataset/

Install opencv

https://www.pyimagesearch.com/2015/07/20/install-opencv-3-0-and-python-3-4-on-ubuntu/

Auto-Encoder

https://github.com/aymericdamien/TensorFlow-Examples/blob/master/notebooks/3_NeuralNetworks/autoencoder.ipynb

awesome python library
https://pypi.python.org/pypi/tqdm

https://github.com/carpedm20/BEGAN-tensorflow/blob/master/models.py

https://arxiv.org/pdf/1703.10717.pdf

loss=nan solution

http://blog.csdn.net/threadroc/article/details/53967560

### Preparing training set

In [2]:
# Preparing training set
import glob
import random
from matplotlib import pyplot as plt
import matplotlib.image as mpimg

class CK:
    
    def __init__(self):
        self.emotions = ["neutral", "anger", "contempt", "disgust", "fear", "happy", "sadness", "surprise"] #Define emotions
        self.file_paths = []
        self.start_of_new_batch = 0
        self.height = 60
        self.width = 60
        self.channels = 1
        self.images_tensor=[]

        for emotion in self.emotions:
            self.file_paths.extend(glob.glob("./Face_data/CK+/dataset/%s/*" %emotion))

        random.shuffle(self.file_paths)
        size = len(self.file_paths)
        # Here generating a tensor of type string that include all the filename with png extention
        filename_queue  = tf.train.string_input_producer(self.file_paths)
        # Initializing a file Reader
        image_reader = tf.WholeFileReader()
        # Get all the files mentioned ie filename queue and
        # returns the  the file name and the pixelvalue in form of a tensor !
        _,imagefile= image_reader.read(filename_queue)
        image = tf.image.decode_png(imagefile, channels=self.channels)
        image.set_shape((self.height,self.width,self.channels))
        images = tf.train.shuffle_batch([image],
                                        batch_size=size,
                                        capacity=size,
                                        min_after_dequeue=0)

        with tf.Session() as sess:
            tf.global_variables_initializer().run()
            # Coordinate the loading of image files.
            coord = tf.train.Coordinator()
            threads = tf.train.start_queue_runners(coord=coord)

            # Get a batch of image tensors.
            self.images_tensor = sess.run(images)

            # Finish off the filename queue coordinator.
            coord.request_stop()
            coord.join(threads)
        
        self.images_tensor = self.images_tensor / 255. 
        
#         img = mpimg.imread(paths_for_this_patch[0])
#         print(paths_for_this_patch[0])
#         im = images_tensor[0][0]
#         plt.figure(1)
#         plt.imshow(im.squeeze(), cmap='gray')
#         plt.figure(2)
#         plt.imshow(img, cmap='gray')
#         plt.show()
        
        
    def next_batch(self, size):
        
        if size > len(self.file_paths):
            size = len(self.file_paths)
        if size <= 0:
            return []
            
        # Get images for this batch
        this_patch = []
        start = self.start_of_new_batch
        end = (start + size - 1) % len(self.file_paths)
        if start > end:
            this_patch = self.images_tensor[start:len(self.file_paths)]
            this_patch = np.concatenate((this_patch,
                                         (self.images_tensor[0:end+1])))
        else:
            this_patch = self.images_tensor[start:end+1]
            
        self.start_of_new_batch = (start + size) % len(self.file_paths)
        

        return  this_patch

train_CK = CK()


### Helper Function

In [3]:
#This function performns a leaky relu activation, which is needed for the discriminator network.
def lrelu(x, leak=0.2, name="lrelu"):
     with tf.variable_scope(name):
         f1 = 0.5 * (1 + leak)
         f2 = 0.5 * (1 - leak)
         return f1 * x + f2 * abs(x)
    
#The below functions are taken from carpdem20's implementation https://github.com/carpedm20/DCGAN-tensorflow
#They allow for saving sample images from the generator to follow progress
def save_images(images, size, image_path):
    return imsave(inverse_transform(images), size, image_path)

def imsave(images, size, path):
    return scipy.misc.imsave(path, merge(images, size))

def inverse_transform(images):
    return (images+1.)/2.

def merge(images, size):
    h, w = images.shape[1], images.shape[2]
    img = np.zeros((h * size[0], w * size[1]))

    for idx, image in enumerate(images):
        i = idx % size[1]
        j = idx // size[1]
        img[j*h:j*h+h, i*w:i*w+w] = image

    return img

def get_shape(tensor): # static shape
    return tensor.get_shape().as_list()

## Defining the Adversarial Networks

### Generator Network

In [49]:
def generator(z):
    
    # size: (out_height/(2*2*2)) * (out_width/(2*2*2)) * n
    zP = slim.fully_connected(z,8*8*256,normalizer_fn=slim.batch_norm,\
        activation_fn=lrelu,scope='g_project',weights_initializer=initializer)
    zCon = tf.reshape(zP,[-1,8,8,256])
    
    gen1 = slim.convolution2d_transpose(\
        zCon,num_outputs=128,kernel_size=[4,4],stride=[2,2],\
        padding="SAME",normalizer_fn=slim.batch_norm,\
        activation_fn=lrelu,scope='g_conv1', weights_initializer=initializer)
    
    gen2 = slim.convolution2d_transpose(\
        gen1,num_outputs=64,kernel_size=[4,4],stride=[2,2],\
        padding="SAME",normalizer_fn=slim.batch_norm,\
        activation_fn=lrelu,scope='g_conv2', weights_initializer=initializer)
    
    gen3 = slim.convolution2d_transpose(\
        gen2,num_outputs=32,kernel_size=[4,4],stride=[2,2],\
        padding="SAME",normalizer_fn=slim.batch_norm,\
        activation_fn=lrelu,scope='g_conv3', weights_initializer=initializer)
    
    g_out = slim.convolution2d_transpose(\
        gen3,num_outputs=1,kernel_size=[64,64],padding="SAME",\
        biases_initializer=None,activation_fn=tf.nn.tanh,\
        scope='g_out', weights_initializer=initializer)
    
    return g_out

### Discriminator Network

The line of concat y with dis1 refer to:
https://github.com/Eyyub/tensorflow-cdcgan/blob/master/model/discriminator.py

Idea comes from IcGAN:
https://arxiv.org/abs/1611.06355

In [50]:
def discriminator(bottom, reuse=False):
    
    dis1 = slim.convolution2d(bottom,64,[4,4],stride=[2,2],padding="SAME",\
        biases_initializer=None,activation_fn=lrelu,\
        reuse=reuse,scope='d_conv1',weights_initializer=initializer)    
    dis2 = slim.convolution2d(dis1,128,[4,4],stride=[2,2],padding="SAME",\
        normalizer_fn=slim.batch_norm,activation_fn=lrelu,\
        reuse=reuse,scope='d_conv2', weights_initializer=initializer)
    dis3 = slim.convolution2d(dis2,256,[4,4],stride=[2,2],padding="SAME",\
        normalizer_fn=slim.batch_norm,activation_fn=lrelu,\
        reuse=reuse,scope='d_conv3',weights_initializer=initializer)
    dis4 = slim.convolution2d(dis3,512,[4,4],stride=[2,2],padding="SAME",\
        normalizer_fn=slim.batch_norm,activation_fn=lrelu,\
        reuse=reuse,scope='d_conv4',weights_initializer=initializer)
    
    d_out = slim.fully_connected(slim.flatten(dis4),1,activation_fn=tf.nn.sigmoid,\
        reuse=reuse,scope='d_out', weights_initializer=initializer)
    
    return d_out

### Connecting them together

In [53]:
tf.reset_default_graph()

z_size = 100 #Size of z vector used for generator.

#This initializaer is used to initialize all the weights of the network.
initializer = tf.truncated_normal_initializer(stddev=0.02)

#These two placeholders are used for input into the generator and discriminator, respectively.
z_in = tf.placeholder(shape=[None,z_size],dtype=tf.float32) #Random vector
real_in = tf.placeholder(shape=[None,64,64,1],dtype=tf.float32) #Real images


Gz = generator(z_in) #Generates images from random z vectors
Dx = discriminator(real_in) #Produces probabilities for real images
Dg = discriminator(Gz, reuse=True) #Produces probabilities for generator images

#These functions together define the optimization objective of the GAN.
# d_loss = -tf.reduce_mean(tf.log(tf.clip_by_value(Dx,1e-10,1.0)) + tf.log(1.-tf.clip_by_value(Dg,1e-10,1.0))) #This optimizes the discriminator.
# g_loss = -tf.reduce_mean(tf.log(tf.clip_by_value(Dg,1e-10,1.0))) #This optimizes the generator.
d_loss = -tf.reduce_mean(tf.log(Dx) + tf.log(1.-Dg)) #This optimizes the discriminator.
g_loss = -tf.reduce_mean(tf.log(Dg)) #This optimizes the generator.

tvars = tf.trainable_variables()

#The below code is responsible for applying gradient descent to update the GAN.
trainerD = tf.train.AdamOptimizer(learning_rate=0.0002,beta1=0.5)
trainerG = tf.train.AdamOptimizer(learning_rate=0.0002,beta1=0.5)
d_grads = trainerD.compute_gradients(d_loss,tvars[9:]) #Only update the weights for the discriminator network.
g_grads = trainerG.compute_gradients(g_loss,tvars[0:9]) #Only update the weights for the generator network.

update_D = trainerD.apply_gradients(d_grads)
update_G = trainerG.apply_gradients(g_grads)

### Training the network

In [58]:
batch_size = 128 #Size of image batch to apply at each iteration.
iterations = 50000 #Total number of iterations to use. 50000 -> 2hrs
sample_size = 4 # Number of sample to be view
sample_directory = './Face_DCGANs_figs' #Directory to save sample images from generator in.
model_directory = './Face_DCGANs_models' #Directory to save trained model to.

init = tf.initialize_all_variables()
saver = tf.train.Saver()
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:  
    sess.run(init)
    for i in range(iterations):
        #zs = np.random.uniform(-1.0,1.0,size=[batch_size,z_size]).astype(np.float32) #Generate a random z batch
        zs = np.random.normal(size=(batch_size, z_size))
        xs = train_CK.next_batch(batch_size) #Draw a sample batch from dataset.
        xs = (np.reshape(xs,[batch_size,60,60,1]) - 0.5) * 2.0 #Transform it to be between -1 and 1
        xs = np.lib.pad(xs, ((0,0),(2,2),(2,2),(0,0)),'constant', constant_values=(-1, -1)) #Pad the images so the are 64x64
        _,dLoss = sess.run([update_D,d_loss],feed_dict={z_in:zs,real_in:xs}) #Update the discriminator
        if i % 2 == 0:
            _,gLoss = sess.run([update_G,g_loss],feed_dict={z_in:zs}) #Update the generator, twice for good measure.
            _,gLoss = sess.run([update_G,g_loss],feed_dict={z_in:zs})
        if i % 10 == 0:
            print("Gen Loss: " + str(gLoss) + " Disc Loss: " + str(dLoss))
            z2 = np.random.uniform(-1.0,1.0,size=[sample_size,z_size]).astype(np.float32) #Generate another z batch
            
            newZ = sess.run(Gz,feed_dict={z_in:z2}) #Use new z to get sample images from generator.
            if not os.path.exists(sample_directory):
                os.makedirs(sample_directory)
            #Save sample generator images for viewing training progress.
            save_images(np.reshape(newZ[0:4],[4,64,64]),[2,2],sample_directory+'/fig'+str(i)+'.png')
        if i % 1000 == 0 and i != 0:
            if not os.path.exists(model_directory):
                os.makedirs(model_directory)
            saver.save(sess,model_directory+'/model-'+str(i)+'.cptk')
            print("Saved Model")

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Gen Loss: 0.474374 Disc Loss: 1.49134
Gen Loss: 2.42464 Disc Loss: 0.0478356
Gen Loss: 6.94997 Disc Loss: 0.0246885
Gen Loss: 7.40649 Disc Loss: 0.0111768
Gen Loss: 6.95707 Disc Loss: 0.00608021
Gen Loss: 7.80961 Disc Loss: 0.00520655
Gen Loss: 6.67809 Disc Loss: 0.00374943
Gen Loss: 5.998 Disc Loss: 0.00320411
Gen Loss: 10.2151 Disc Loss: 0.00480482
Gen Loss: 10.4406 Disc Loss: 0.00187392
Gen Loss: 9.43026 Disc Loss: 0.00212386
Gen Loss: 8.63427 Disc Loss: 0.00129623
Gen Loss: 8.23524 Disc Loss: 0.00200823
Gen Loss: 9.59312 Disc Loss: 0.00173623
Gen Loss: 7.90111 Disc Loss: 0.00157419
Gen Loss: 9.04184 Disc Loss: 0.00107176
Gen Loss: 6.28197 Disc Loss: 0.00127906
Gen Loss: 7.30457 Disc Loss: 0.0014044
Gen Loss: 7.85243 Disc Loss: 0.00153004
Gen Loss: 6.19726 Disc Loss: 0.000901374
Gen Loss: 9.40575 Disc Loss: 0.00116119
Gen Loss: 10.8523 Disc Loss: 0.00124411
Gen Loss: 7.74235 Disc Loss: 0.00123962
Gen Loss: 8.6

Gen Loss: 10.8764 Disc Loss: 0.000121261
Gen Loss: 10.9777 Disc Loss: 4.93967e-05
Gen Loss: 11.5192 Disc Loss: 6.16312e-05
Gen Loss: 10.5815 Disc Loss: 5.23875e-05
Gen Loss: 12.1294 Disc Loss: 5.81407e-05
Gen Loss: 11.1949 Disc Loss: 9.34898e-05
Gen Loss: 11.2224 Disc Loss: 9.11425e-05
Gen Loss: 11.5113 Disc Loss: 8.47562e-05
Saved Model
Gen Loss: 10.2995 Disc Loss: 4.91452e-05
Gen Loss: 10.515 Disc Loss: 2.42213e-05
Gen Loss: 11.2638 Disc Loss: 1.161e-05
Gen Loss: 9.80352 Disc Loss: 9.5321e-05
Gen Loss: 10.7467 Disc Loss: 6.65441e-05
Gen Loss: 9.14486 Disc Loss: 3.75824e-05
Gen Loss: 13.4981 Disc Loss: 1.64658e-05
Gen Loss: 9.00276 Disc Loss: 2.23388e-05
Gen Loss: 10.4525 Disc Loss: 6.0004e-05
Gen Loss: 8.39618 Disc Loss: 2.0929e-05
Gen Loss: 8.82989 Disc Loss: 8.0756e-05
Gen Loss: 10.3287 Disc Loss: 1.81689e-05
Gen Loss: 12.2856 Disc Loss: 2.76691e-05
Gen Loss: 8.97308 Disc Loss: 5.56765e-05
Gen Loss: 11.379 Disc Loss: 8.69402e-05
Gen Loss: 12.6465 Disc Loss: 3.45922e-05
Gen Loss: 11

Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan Disc Loss: nan
Gen Loss: nan 

KeyboardInterrupt: 

### Using a trained network